In [1]:
!pip install pandas numpy scikit-learn scipy joblib


In [2]:
import os
import gzip
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.stats import skew, kurtosis

In [23]:
# 1. Load and preprocess data
def load_data(file_path):
    """Load a compressed CSV file and return a pandas DataFrame."""
    with gzip.open(file_path, 'rt') as f:
        return pd.read_csv(f)

def preprocess_data(df):
    """Preprocess the DataFrame by selecting relevant features and handling missing values."""
    # Select sensor columns (accelerometer, gyroscope, magnetometer)
    sensor_cols = [col for col in df.columns if col.startswith(('raw_acc:', 'proc_gyro:', 'raw_magnet:'))]
    selected_data = df[sensor_cols]

    # Handle missing values (e.g., fill with zeros or interpolate)
    selected_data.fillna(0, inplace=True)

    # Filter for target labels only
    target_labels = [
        'label:SITTING', 'label:FIX_walking', 'label:BICYCLING', 'label:FIX_running',
        'label:STAIRS_-_GOING_UP', 'label:STAIRS_-_GOING_DOWN'
    ]
    labels = df[target_labels].dropna(how='all')

    return selected_data, labels

In [24]:
# 2. Feature extraction
'''Extracts statistical features (mean, standard deviation, skewness, kurtosis, min, max) from a sliding window of data.
Produces a feature vector summarizing each window.'''

def extract_features(window):
    """Extract statistical features from a window of data."""
    features = {}
    for col in window.columns:
        features[f'{col}_mean'] = window[col].mean()
        features[f'{col}_std'] = window[col].std()
        features[f'{col}_min'] = window[col].min()
        features[f'{col}_max'] = window[col].max()
        features[f'{col}_skew'] = skew(window[col])
        features[f'{col}_kurtosis'] = kurtosis(window[col])
    return features

In [38]:
'''Creates a feature dataset by: Dividing the data into overlapping windows, Extracting features for each window.
   Assigning the most frequent label within the window as the target label.
   Output: A feature matrix X and label vector y'''

def create_feature_dataset(data, labels, window_size=50, step_size=25):
    """Create a dataset of features and corresponding labels from sliding windows."""
    X, y = [], []
    for start in range(0, len(data) - window_size, step_size):
        end = start + window_size
        window = data.iloc[start:end]
        label_window = labels.iloc[start:end]

        # Extract features from the window
        features = extract_features(window)
        X.append(features)

        # Calculate mode for each label column in the window
        label_modes = label_window.mode()
        if not label_modes.empty:
            label_modes = label_modes.iloc[0]  # Take the first mode if multiple exist
            label = label_modes.idxmax()  # Choose the label with the highest occurrence
            y.append(label)
        else:
            y.append(np.nan)  # Assign NaN if no mode is available

    # Remove invalid rows
    X = pd.DataFrame(X)
    y = pd.Series(y)

    valid_indices = ~y.isna()
    X = X[valid_indices]
    y = y[valid_indices]

    return X, y




In [39]:
# 3. Model training and evaluation
def train_model(X, y):
    """Train a Random Forest classifier and return the trained model."""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the model
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Evaluate the model
    y_pred = clf.predict(X_test)
    print("\nClassification Report:\n")
    print(classification_report(y_test, y_pred))

    return clf

In [40]:
# 4. Save the model using joblib for deployment
import joblib

def save_model(model, output_path):
    """Save the trained model to a file."""
    joblib.dump(model, output_path) #write the trained model to a file

In [19]:
# Upload dataset folder to Colab
from google.colab import files
import zipfile
import shutil

print("Please upload your dataset folder (compressed as a zip file).")
uploaded = files.upload()


data_folder = 'data'
with zipfile.ZipFile(next(iter(uploaded.keys())), 'r') as zip_ref:
    zip_ref.extractall(data_folder)

Please upload your dataset folder (compressed as a zip file).


KeyboardInterrupt: 

In [41]:
# Main workflow
data_folder_path = os.path.join(data_folder)  # Path to extracted folder
all_files = [os.path.join(data_folder_path, file) for file in os.listdir(data_folder_path) if file.endswith('.csv.gz')]

In [42]:
print("Columns in the dataset:")
print(df.columns.tolist())


Columns in the dataset:
['timestamp', 'raw_acc:magnitude_stats:mean', 'raw_acc:magnitude_stats:std', 'raw_acc:magnitude_stats:moment3', 'raw_acc:magnitude_stats:moment4', 'raw_acc:magnitude_stats:percentile25', 'raw_acc:magnitude_stats:percentile50', 'raw_acc:magnitude_stats:percentile75', 'raw_acc:magnitude_stats:value_entropy', 'raw_acc:magnitude_stats:time_entropy', 'raw_acc:magnitude_spectrum:log_energy_band0', 'raw_acc:magnitude_spectrum:log_energy_band1', 'raw_acc:magnitude_spectrum:log_energy_band2', 'raw_acc:magnitude_spectrum:log_energy_band3', 'raw_acc:magnitude_spectrum:log_energy_band4', 'raw_acc:magnitude_spectrum:spectral_entropy', 'raw_acc:magnitude_autocorrelation:period', 'raw_acc:magnitude_autocorrelation:normalized_ac', 'raw_acc:3d:mean_x', 'raw_acc:3d:mean_y', 'raw_acc:3d:mean_z', 'raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'raw_acc:3d:std_z', 'raw_acc:3d:ro_xy', 'raw_acc:3d:ro_xz', 'raw_acc:3d:ro_yz', 'proc_gyro:magnitude_stats:mean', 'proc_gyro:magnitude_stats:std', '

In [43]:
# Load and process all files

all_features, all_labels = [], []
for file in all_files:
    print(f"Processing file: {file}")
    df = load_data(file)
    print("Data loaded successfully. Preview:")
    print(df.head())

    data, labels = preprocess_data(df)
    print("Preprocessed data preview:")
    print(data.head())
    print("Labels preview:")
    print(labels.head())

    # Extract features and labels
    features, label_series = create_feature_dataset(data, labels)
    print("Extracted features preview:")
    print(features.head())
    print("Extracted labels preview:")
    print(label_series.head())

    all_features.append(features)
    all_labels.append(label_series)



Processing file: data/F50235E0-DD67-4F2A-B00B-1F31ADA998B9.features_labels.csv.gz
Data loaded successfully. Preview:
    timestamp  raw_acc:magnitude_stats:mean  raw_acc:magnitude_stats:std  \
0  1441900642                      1.049387                     0.031372   
1  1441900950                      1.053630                     0.093730   
2  1441901019                      1.043679                     0.009139   
3  1441901075                      1.048666                     0.006735   
4  1441901135                      1.045069                     0.020269   

   raw_acc:magnitude_stats:moment3  raw_acc:magnitude_stats:moment4  \
0                         0.037631                         0.070507   
1                         0.119303                         0.212048   
2                         0.003037                         0.012389   
3                        -0.006034                         0.010480   
4                        -0.018300                         0.052268   


<ipython-input-23-3799c639d0d9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.fillna(0, inplace=True)


Extracted features preview:
   raw_acc:magnitude_stats:mean_mean  raw_acc:magnitude_stats:mean_std  \
0                           1.054411                          0.011782   
1                           1.057188                          0.011785   
2                           1.055425                          0.008727   
3                           1.056007                          0.006763   
4                           1.053814                          0.002801   

   raw_acc:magnitude_stats:mean_min  raw_acc:magnitude_stats:mean_max  \
0                          1.037134                          1.086594   
1                          1.026043                          1.086594   
2                          1.026043                          1.089851   
3                          1.045457                          1.089851   
4                          1.047103                          1.061873   

   raw_acc:magnitude_stats:mean_skew  raw_acc:magnitude_stats:mean_kurtosis  \
0        

<ipython-input-23-3799c639d0d9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.fillna(0, inplace=True)
<ipython-input-24-81cde4a3a539>:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[f'{col}_skew'] = skew(window[col])
<ipython-input-24-81cde4a3a539>:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[f'{col}_kurtosis'] = kurtosis(window[col])
<ipython-input-24-81cde4a3a539>:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical.

Extracted features preview:
   raw_acc:magnitude_stats:mean_mean  raw_acc:magnitude_stats:mean_std  \
0                           1.010013                          0.047497   
1                           0.997961                          0.012204   
2                           1.003648                          0.020034   
3                           1.015234                          0.025674   
4                           1.009371                          0.021889   

   raw_acc:magnitude_stats:mean_min  raw_acc:magnitude_stats:mean_max  \
0                          0.975364                          1.267366   
1                          0.975364                          1.033370   
2                          0.988482                          1.124153   
3                          0.986149                          1.125149   
4                          0.986149                          1.125149   

   raw_acc:magnitude_stats:mean_skew  raw_acc:magnitude_stats:mean_kurtosis  \
0        

...

Extracted features preview:
   raw_acc:magnitude_stats:mean_mean  raw_acc:magnitude_stats:mean_std  \
0                           1.010259                          0.015712   
1                           1.005115                          0.017208   
2                           0.996101                          0.002640   
3                           1.001512                          0.018723   
4                           1.023354                          0.048337   

   raw_acc:magnitude_stats:mean_min  raw_acc:magnitude_stats:mean_max  \
0                          0.996766                          1.103694   
1                          0.993801                          1.103694   
2                          0.990713                          1.001872   
3                          0.988029                          1.063154   
4                          0.984494                          1.136571   

   raw_acc:magnitude_stats:mean_skew  raw_acc:magnitude_stats:mean_kurtosis  \
0        

<ipython-input-23-3799c639d0d9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.fillna(0, inplace=True)


Extracted features preview:
   raw_acc:magnitude_stats:mean_mean  raw_acc:magnitude_stats:mean_std  \
0                           0.938653                          0.051406   
1                           0.936718                          0.051820   
2                           0.927496                          0.005440   
3                           0.947266                          0.058553   
4                           0.943957                          0.059499   

   raw_acc:magnitude_stats:mean_min  raw_acc:magnitude_stats:mean_max  \
0                          0.921301                          1.230308   
1                          0.921301                          1.230308   
2                          0.921523                          0.957338   
3                          0.926824                          1.215584   
4                          0.919798                          1.215584   

   raw_acc:magnitude_stats:mean_skew  raw_acc:magnitude_stats:mean_kurtosis  \
0        

<ipython-input-23-3799c639d0d9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data.fillna(0, inplace=True)
<ipython-input-24-81cde4a3a539>:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[f'{col}_skew'] = skew(window[col])
<ipython-input-24-81cde4a3a539>:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[f'{col}_kurtosis'] = kurtosis(window[col])
<ipython-input-24-81cde4a3a539>:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical.

Extracted features preview:
   raw_acc:magnitude_stats:mean_mean  raw_acc:magnitude_stats:mean_std  \
0                           1.008035                          0.023525   
1                           1.013052                          0.028418   
2                           1.002735                          0.023046   
3                           0.999832                          0.013167   
4                           1.004327                          0.023006   

   raw_acc:magnitude_stats:mean_min  raw_acc:magnitude_stats:mean_max  \
0                          0.992792                          1.064523   
1                          0.990643                          1.095367   
2                          0.990298                          1.095367   
3                          0.990298                          1.052843   
4                          0.991522                          1.153102   

   raw_acc:magnitude_stats:mean_skew  raw_acc:magnitude_stats:mean_kurtosis  \
0        

In [44]:
# Combine data from all participants
X = pd.concat(all_features, ignore_index=True)
y = pd.concat(all_labels, ignore_index=True)
print("Combined features and labels:")
print(X.head())
print(y.head())



Combined features and labels:
   raw_acc:magnitude_stats:mean_mean  raw_acc:magnitude_stats:mean_std  \
0                           1.054411                          0.011782   
1                           1.057188                          0.011785   
2                           1.055425                          0.008727   
3                           1.056007                          0.006763   
4                           1.053814                          0.002801   

   raw_acc:magnitude_stats:mean_min  raw_acc:magnitude_stats:mean_max  \
0                          1.037134                          1.086594   
1                          1.026043                          1.086594   
2                          1.026043                          1.089851   
3                          1.045457                          1.089851   
4                          1.047103                          1.061873   

   raw_acc:magnitude_stats:mean_skew  raw_acc:magnitude_stats:mean_kurtosis  \
0      

In [45]:
# Train the model
print("\nTraining the model...")
model = train_model(X, y)



Training the model...

Classification Report:

                         precision    recall  f1-score   support

        label:BICYCLING       0.00      0.00      0.00        31
      label:FIX_running       0.00      0.00      0.00         7
      label:FIX_walking       0.50      0.02      0.04        95
          label:SITTING       0.95      1.00      0.97      2329
label:STAIRS_-_GOING_UP       0.00      0.00      0.00         1

               accuracy                           0.95      2463
              macro avg       0.29      0.20      0.20      2463
           weighted avg       0.91      0.95      0.92      2463



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [46]:
# Save the model for deployment
save_model(model, 'movement_detection_model.pkl')
print("Model saved as 'movement_detection_model.pkl'")


Model saved as 'movement_detection_model.pkl'


In [47]:
# Download the model
from google.colab import files
files.download('movement_detection_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>